## Silver data analysis

Based on the data for the refractive index for silver obtained from [Johnson and Christy, 1972](http://fisica.ufpr.br/jfreire/EstadoSolido/optical%20contstants%20of%20noble%20metals.pdf) we dicide to dig deeper on the option for interpolation. 

* Data interpolation decision
* Experimental data analysis 

### Data interpolation desicion

We try linear and a spline interpolation. We use the module `scipy.interpolate` in particular [`interp1d`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.interpolate.interp1d.html) for the linear interpolation and, [`splrep`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.splrep.html) and [`splev`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.splev.html) for the splines .